In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import transformers
from IPython.display import display
from sklearn.utils import shuffle
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import glob
import json
import os
from typing import Optional, Tuple

2024-06-20 19:43:17.343991: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 19:43:17.353324: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 19:43:17.726881: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 19:43:18.739391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 19:43:22.288507: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [3]:
#df = pd.read_csv('../data/10k_preproc')

In [30]:
DATA_PATH = "../raw_data/AI4Code"
BASE_MODEL = "../models/distilbert-base-uncased"
N_SPLITS = 5
SEQ_LEN = 128
RANDOM_STATE = 42

try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    STRATEGY = tf.distribute.experimental.TPUStrategy(TPU)
    BATCH_SIZE = 128 * STRATEGY.num_replicas_in_sync
except Exception:
    TPU = None
    STRATEGY = tf.distribute.get_strategy()
    BATCH_SIZE = 32
    LIMIT = 10_000

print("TensorFlow", tf.__version__)

if TPU is not None:
    print("Using TPU v3-8")
else:
    print("Using GPU/CPU")

print("Batch size:", BATCH_SIZE)

TensorFlow 2.16.1
Using GPU/CPU
Batch size: 32


In [5]:
def read_notebook(path: str) -> pd.DataFrame:
    with open(path) as file:
        df = pd.DataFrame(json.load(file))
    df["id"] = os.path.splitext(os.path.basename(path))[0]
    return df

In [6]:
def expand_order(row: Tuple[str, str]) -> pd.DataFrame:
    cell_ids = row[1].split(" ")
    df = pd.DataFrame(
        {
            "id": [row[0] for _ in range(len(cell_ids))],
            "cell_id": cell_ids,
            "rank": range(len(cell_ids)),
        }
    )
    df["pct_rank"] = df["rank"] / len(df)
    return df

In [7]:
def tokenize(source: pd.Series) -> Tuple[np.array, np.array]:
    tokenizer = transformers.AutoTokenizer.from_pretrained(BASE_MODEL, do_lower_case=True)

    input_ids = np.zeros((len(source), SEQ_LEN), dtype="int32")
    attention_mask = np.zeros((len(source), SEQ_LEN), dtype="int32")

    for i, x in enumerate(tqdm(source, total=len(source))):
        encoding = tokenizer.encode_plus(
            x,
            None,
            add_special_tokens=True,
            max_length=SEQ_LEN,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True,
        )
        input_ids[i] = encoding["input_ids"]
        attention_mask[i] = encoding["attention_mask"]

    return input_ids, attention_mask

In [8]:
def get_dataset(
    input_ids: np.array,
    attention_mask: np.array,
    labels: Optional[np.array] = None,
    ordered: bool = False,
    repeated: bool = False,
) -> tf.data.Dataset:
    if labels is not None:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({"input_ids": input_ids, "attention_mask": attention_mask}, labels)
        )
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            {"input_ids": input_ids, "attention_mask": attention_mask}
        )
    if repeated:
        dataset = dataset.repeat()
    if not ordered:
        dataset = dataset.shuffle(1024)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [9]:
def get_model() -> tf.keras.Model:
    backbone = transformers.TFDistilBertModel.from_pretrained(BASE_MODEL)
    input_ids = tf.keras.layers.Input(
        shape=(SEQ_LEN,),
        dtype=tf.int32,
        name="input_ids",
    )
    attention_mask = tf.keras.layers.Input(
        shape=(SEQ_LEN,),
        dtype=tf.int32,
        name="attention_mask",
    )
    x = backbone(
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
        },
    )
    outputs = tf.keras.layers.Dense(1, activation="linear", dtype="float32")(x[0][:, 0, :])

    model = tf.keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=outputs,
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.MeanSquaredError(),
    )
    return model

# Data Preparation 

In [10]:
paths = glob.glob(os.path.join(DATA_PATH, "train_data", "*.json"))
if LIMIT is not None:
    paths = paths[:LIMIT]

source_df = pd.concat([read_notebook(x) for x in tqdm(paths, total=len(paths))])

source_df = source_df[source_df["cell_type"] == "markdown"]
source_df = source_df.drop("cell_type", axis=1)
source_df = source_df.rename_axis("cell_id").reset_index()

order_df = pd.read_csv(os.path.join(DATA_PATH, "train_orders.csv"), index_col="id")
order_df = pd.concat(
    [expand_order(row) for row in tqdm(order_df.itertuples(), total=len(order_df))])

ancestors_df = pd.read_csv(
    os.path.join(DATA_PATH, "train_ancestors.csv"),
    usecols=["id", "ancestor_id"],
    index_col="id")

df = source_df.merge(order_df, on=["id", "cell_id"]).merge(ancestors_df, on="id")
df = df.dropna()
display(df)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/139256 [00:00<?, ?it/s]

,cell_id,source,id,rank,pct_rank,ancestor_id
0,58900633,#### Explore airports\nThere are 268 unique ai...,2ac1be019bf73e,26,0.440678,b66b5e9a
1,950a8058,#### Airlines\nAfter looking into delay distri...,2ac1be019bf73e,21,0.355932,b66b5e9a
2,3d3eb7f6,Distribution of airlines is extremely right sk...,2ac1be019bf73e,25,0.423729,b66b5e9a
3,99385c1e,#### How did carriers performed over the years?,2ac1be019bf73e,52,0.881356,b66b5e9a
4,26c65ce6,An average departure delay and its std are slo...,2ac1be019bf73e,45,0.762712,b66b5e9a
...,...,...,...,...,...,...
156707,7fafd119,# 라이브러리 & 데이터 불러오기,ff2cfc75d39200,75,0.669643,7cc543d3
156708,e75ce892,## ❔❓ 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기,ff2cfc75d39200,109,0.973214,7cc543d3
156709,99e99cc4,### 정규화,ff2cfc75d39200,100,0.892857,7cc543d3
156710,c63df0e4,# 💓🐨 연습_02 🐨💓,ff2cfc75d39200,108,0.964286,7cc543d3


In [13]:
lang_df = pd.read_csv('../raw_data/AI4Code/all_languages.csv')
merged_df = df.merge(lang_df, on='id', how='left')
merged_df

,cell_id,source,id,rank,pct_rank,ancestor_id,language,score
0,58900633,#### Explore airports\nThere are 268 unique ai...,2ac1be019bf73e,26,0.440678,b66b5e9a,en,0.999996
1,950a8058,#### Airlines\nAfter looking into delay distri...,2ac1be019bf73e,21,0.355932,b66b5e9a,en,0.999996
2,3d3eb7f6,Distribution of airlines is extremely right sk...,2ac1be019bf73e,25,0.423729,b66b5e9a,en,0.999996
3,99385c1e,#### How did carriers performed over the years?,2ac1be019bf73e,52,0.881356,b66b5e9a,en,0.999996
4,26c65ce6,An average departure delay and its std are slo...,2ac1be019bf73e,45,0.762712,b66b5e9a,en,0.999996
...,...,...,...,...,...,...,...,...
156707,7fafd119,# 라이브러리 & 데이터 불러오기,ff2cfc75d39200,75,0.669643,7cc543d3,en,0.428570
156708,e75ce892,## ❔❓ 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기,ff2cfc75d39200,109,0.973214,7cc543d3,en,0.428570
156709,99e99cc4,### 정규화,ff2cfc75d39200,100,0.892857,7cc543d3,en,0.428570
156710,c63df0e4,# 💓🐨 연습_02 🐨💓,ff2cfc75d39200,108,0.964286,7cc543d3,en,0.428570


In [18]:
merged_df = merged_df[merged_df['score'] >= 0.75]
merged_df = merged_df[merged_df['language'] == 'en']
merged_df

,cell_id,source,id,rank,pct_rank,ancestor_id,language,score
0,58900633,#### Explore airports\nThere are 268 unique ai...,2ac1be019bf73e,26,0.440678,b66b5e9a,en,0.999996
1,950a8058,#### Airlines\nAfter looking into delay distri...,2ac1be019bf73e,21,0.355932,b66b5e9a,en,0.999996
2,3d3eb7f6,Distribution of airlines is extremely right sk...,2ac1be019bf73e,25,0.423729,b66b5e9a,en,0.999996
3,99385c1e,#### How did carriers performed over the years?,2ac1be019bf73e,52,0.881356,b66b5e9a,en,0.999996
4,26c65ce6,An average departure delay and its std are slo...,2ac1be019bf73e,45,0.762712,b66b5e9a,en,0.999996
...,...,...,...,...,...,...,...,...
156679,d5e9c516,# Optimization,d3c351143d72ef,36,0.418605,77eaf8c7,en,0.999995
156680,47fd56dd,# Optimizer Analytics,d3c351143d72ef,28,0.325581,77eaf8c7,en,0.999995
156681,a3bdddf7,# Define Constraints,d3c351143d72ef,10,0.116279,77eaf8c7,en,0.999995
156682,67b26d0c,In validation RMS 99.4 and Adam 66.4 this crea...,d3c351143d72ef,46,0.534884,77eaf8c7,en,0.999995


In [25]:
merged_df = merged_df.drop(columns=['language', 'score'])
merged_df

KeyError: "['language', 'score'] not found in axis"

In [26]:
file_path = os.path.join(DATA_PATH, "merged_data.csv")
merged_df.to_csv(file_path, index=False)

In [27]:
merged_df

,cell_id,source,id,rank,pct_rank,ancestor_id
0,58900633,#### Explore airports\nThere are 268 unique ai...,2ac1be019bf73e,26,0.440678,b66b5e9a
1,950a8058,#### Airlines\nAfter looking into delay distri...,2ac1be019bf73e,21,0.355932,b66b5e9a
2,3d3eb7f6,Distribution of airlines is extremely right sk...,2ac1be019bf73e,25,0.423729,b66b5e9a
3,99385c1e,#### How did carriers performed over the years?,2ac1be019bf73e,52,0.881356,b66b5e9a
4,26c65ce6,An average departure delay and its std are slo...,2ac1be019bf73e,45,0.762712,b66b5e9a
...,...,...,...,...,...,...
156679,d5e9c516,# Optimization,d3c351143d72ef,36,0.418605,77eaf8c7
156680,47fd56dd,# Optimizer Analytics,d3c351143d72ef,28,0.325581,77eaf8c7
156681,a3bdddf7,# Define Constraints,d3c351143d72ef,10,0.116279,77eaf8c7
156682,67b26d0c,In validation RMS 99.4 and Adam 66.4 this crea...,d3c351143d72ef,46,0.534884,77eaf8c7


In [33]:
from transformers import TFAutoModel, AutoTokenizer

model_name = "distilbert-base-uncased"

model = TFAutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

BASE_MODEL = "../models/distilbert-base-uncased"
model.save_pretrained(BASE_MODEL)
tokenizer.save_pretrained(BASE_MODEL)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

('../models/distilbert-base-uncased/tokenizer_config.json',
 '../models/distilbert-base-uncased/special_tokens_map.json',
 '../models/distilbert-base-uncased/vocab.txt',
 '../models/distilbert-base-uncased/added_tokens.json',
 '../models/distilbert-base-uncased/tokenizer.json')

In [31]:
input_ids, attention_mask = tokenize(merged_df["source"])

labels = merged_df["pct_rank"].to_numpy()
groups = merged_df["ancestor_id"].to_numpy()

print("input_ids:", input_ids.shape)
print("attention_mask:", attention_mask.shape)
print("labels:", labels.shape)
print("groups:", groups.shape)

OSError: ../models/distilbert-base-uncased does not appear to have a file named config.json. Checkout 'https://huggingface.co/../models/distilbert-base-uncased/tree/None' for available files.

In [ ]:
from transformers import TFAutoModel, AutoTokenizer
